Note: This jupyter notebook was adapted from my initial work done on a Google Collab space, therefore some implementations reflect that.

### Installations

The following are the necessary installations required to run this notebook.

Make sure you have the `requirements.txt` file!!!

Change the path accordingly to where your file is stored

#### Google Collab Steps

**Mounting Google Drive**

If on a Google Collab space, ensure to mount your Google drive to the Jupyter Notebook.

```python
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
```

**Install Requirements**

```python
%pip install -qU -r /content/requirements.txt
```

#### Launching Xterm

The following are steps that will be used to launch xterm